CodeStates Deep Learning Project

---

# Part0. 프로젝트 개요

## 썸네일 자동 생성을 위한 유트브 썸네일 학습

만들고자 하는 서비스는 영상을 분석하여 영상에 어울리는 썸네일을 자동으로 제작해 주는 것입니다.<br/>
썸네일에는 아래 두 가지 옵션이 들어갈 것입니다.

> 1. 해당 영상의 카테고리에 맞는 영상 프레임 중 한 장면
> 2. 영상의 제목

따라서 이 프로젝트의 파이프라인은 아래와 같습니다.
> 1. 데이터 수집
> 2. 데이터 전처리
> 3. 썸네일 학습
> 4. 영상 프레임 중 학습된 이미지와 가장 비슷한 이미지 추출
> 5. 영상 제목(혹은 제목의 일부) 썸네일에 생성
> 6. 영상 혹은 영상 제목과 관련된 작은 이미지 추가


## Pipeline 설명

### 1. 데이터
데이터는 지난 머신러닝 프로젝트에서 사용했던 데이터를 사용할 것입니다.<br/>
https://www.kaggle.com/datasets/datasnaek/youtube-new<br/>
이 데이터 셋은 총 10개 국가의 유튜브 Trending(인기영상)에 오른 영상 데이터입니다.<br/>
Thumbnail 이미지에 대한 정보가 있으므로 이 데이터 셋의 썸네일 이미지를 사용할 것입니다.

### 2. 데이터 전처리
학습에 사용할 썸네일은 학습 시간을 고려하여, 조회수 순으로 순위를 매겨 데이터 크기를 적절히 조절할 것입니다.<br/>
> 1. 데이터는 카테고리 아이디와 썸네일 링크만을 사용할 것입니다.
> 2. 카테고리별로 알맞은 썸네일을 학습하기 위해 데이터 셋을 나누어 줍니다.
> 3. 특정 카테고리의 데이터 수가 적을 경우, 이미지 증강을 사용하여 데이터 수를 늘려줍니다.<br/>

#### 이 프로젝트에서는 Entertainment 카테고리에 있는 영상 일부만을 훈련하고 학습하여 테스트 할 것입니다.

### 3. 썸네일 학습
학습은 AutoEncoder로 실제 이미지를 바탕으로 학습하여 이미지를 생성할 것입니다.<br/>
생성된 이미지는 추후 프레임 내에서 비슷한 이미지를 추출할 때 사용됩니다.

### 4. 유사도 계산
학습된 모델의 Encoder 부분의 잠재벡터와, 프레임의 잠재벡터를 추출하여 유사도 계산을 할 것입니다.

### 4. 썸네일 이미지 추출
가장 유사한 k개의 이미지를 추출하여 사용자에게 프레임을 추천합니다.

### 5. 영상 제목과 이미지 스티커
이 프로젝트에서는 하지 않습니다.

---

## 1. Load Data
1. 데이터 셋 로드
2. 국가코드 기입
3. 사용할 컬럼을 제외한 컬럼 삭제
4. 국가별 카테고리 기입(29번 카테고리 삭제)
5. 썸네일 이미지 불러오기 - 카테고리 별로 이미지 따로 저장

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. 데이터 셋 로드

import pandas as pd

vedios_US = pd.read_csv('/content/drive/MyDrive/project/USvideos.csv')
vedios_CA = pd.read_csv('/content/drive/MyDrive/project/CAvideos.csv')
vedios_GB = pd.read_csv('/content/drive/MyDrive/project/GBvideos.csv')
vedios_IN = pd.read_csv('/content/drive/MyDrive/project/INvideos.csv')
vedios_FR = pd.read_csv('/content/drive/MyDrive/project/FRvideos.csv')
vedios_JP = pd.read_csv('/content/drive/MyDrive/project/JPvideos.csv')
vedios_KR = pd.read_csv('/content/drive/MyDrive/project/KRvideos.csv')
vedios_DE = pd.read_csv('/content/drive/MyDrive/project/DEvideos.csv')
vedios_RU = pd.read_csv('/content/drive/MyDrive/project/RUvideos.csv')
vedios_MX = pd.read_csv('/content/drive/MyDrive/project/MXvideos.csv')

In [ ]:
# 2. 국가 코드 기입

vedios_US['Nation'] = "US"
vedios_CA['Nation'] = "CA"
vedios_GB['Nation'] = "GB"
vedios_IN['Nation'] = "IN"
vedios_FR['Nation'] = "FR"
vedios_JP['Nation'] = "JP"
vedios_KR['Nation'] = "KR"
vedios_DE['Nation'] = "DE"
vedios_RU['Nation'] = "RU"
vedios_MX['Nation'] = "MX"

In [ ]:
# 2-1. 오리지널 데이터 셋 저장

original_dataset = pd.concat([vedios_US, vedios_CA, vedios_GB, vedios_IN, vedios_FR, vedios_JP, vedios_KR, vedios_DE, vedios_RU, vedios_MX]).reset_index(drop=True)
df_original = original_dataset.copy()
df = original_dataset.copy()

In [ ]:
df_original

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Nation
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",US
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,US
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,US
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,17,2018-06-13T13:11:56.000Z,"Canal|""Oficial""|""da""|""Federação""|""Portuguesa""|...",653114,65639,547,3826,https://i.ytimg.com/vi/r63VBOagGAo/default.jpg,False,False,False,Portugal's Football Team Official SongLyrics: ...,MX
375938,i7r_kMbyngk,18.14.06,AMLO llegó con su esposa al Tercer Debate en M...,AMX,25,2018-06-13T01:15:08.000Z,"AMLO|""morena""|""política""|""PRI""|""PAN""|""PRD""|""Ló...",121674,3558,58,491,https://i.ytimg.com/vi/i7r_kMbyngk/default.jpg,False,False,False,AMLO llegó con su esposa al Tercer Debate en M...,MX
375939,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii ¿adorable o ridícula?,Historias Wink,24,2018-06-13T15:08:53.000Z,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,https://i.ytimg.com/vi/_jnwjdMe3Zo/default.jpg,False,False,False,Maire usa una blusa kawaii de Alicia en el paí...,MX
375940,pAH9omNAWA4,18.14.06,La Jefa del Campeón - Capítulo 2 Parte 3/4,Tele Time,22,2018-06-13T02:22:33.000Z,[none],22898,94,11,3,https://i.ytimg.com/vi/pAH9omNAWA4/default.jpg,False,False,False,NaN,MX


In [ ]:
# 3. 국가별 카테고리 기입

def category(file):
    
    import json
    
    df.category_id = df.category_id.astype("str")
    nation = file[31:33]
    
    with open(file, 'r') as f:
        categories = json.load(f)
        
    dic = {}
    
    for category_id in categories['items']:
        dic[category_id['id']] = category_id['snippet']['title']
    
    idx = df.loc[df.Nation == nation].category_id.index
        
    df.category_id[idx] = df.category_id[idx].replace(dic)

In [ ]:
category('/content/drive/MyDrive/project/CA_category_id.json')
category('/content/drive/MyDrive/project/US_category_id.json')
category('/content/drive/MyDrive/project/FR_category_id.json')
category('/content/drive/MyDrive/project/GB_category_id.json')
category('/content/drive/MyDrive/project/JP_category_id.json')
category('/content/drive/MyDrive/project/KR_category_id.json')
category('/content/drive/MyDrive/project/IN_category_id.json')
category('/content/drive/MyDrive/project/DE_category_id.json')
category('/content/drive/MyDrive/project/MX_category_id.json')
category('/content/drive/MyDrive/project/RU_category_id.json')

In [ ]:
df

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Nation
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",US
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,US
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,US
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,Sports,2018-06-13T13:11:56.000Z,"Canal|""Oficial""|""da""|""Federação""|""Portuguesa""|...",653114,65639,547,3826,https://i.ytimg.com/vi/r63VBOagGAo/default.jpg,False,False,False,Portugal's Football Team Official SongLyrics: ...,MX
375938,i7r_kMbyngk,18.14.06,AMLO llegó con su esposa al Tercer Debate en M...,AMX,News & Politics,2018-06-13T01:15:08.000Z,"AMLO|""morena""|""política""|""PRI""|""PAN""|""PRD""|""Ló...",121674,3558,58,491,https://i.ytimg.com/vi/i7r_kMbyngk/default.jpg,False,False,False,AMLO llegó con su esposa al Tercer Debate en M...,MX
375939,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii ¿adorable o ridícula?,Historias Wink,Entertainment,2018-06-13T15:08:53.000Z,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,https://i.ytimg.com/vi/_jnwjdMe3Zo/default.jpg,False,False,False,Maire usa una blusa kawaii de Alicia en el paí...,MX
375940,pAH9omNAWA4,18.14.06,La Jefa del Campeón - Capítulo 2 Parte 3/4,Tele Time,People & Blogs,2018-06-13T02:22:33.000Z,[none],22898,94,11,3,https://i.ytimg.com/vi/pAH9omNAWA4/default.jpg,False,False,False,NaN,MX


In [ ]:
# 5. 중복치 제거

df = df.drop_duplicates(subset="video_id")
df_copy = df.copy()

In [ ]:
# 6. 사용할 컬럼을 제외한 컬럼 삭제: category_id가 29인 Data도 삭제

df_Analysis = df[['Nation', 'category_id', 'views', 'likes', 'comment_count', 'thumbnail_link']].reset_index(drop=True)
df_Analysis = df_Analysis.sort_values('category_id').reset_index(drop=True)
df_Analysis = df_Analysis.loc[1974:]

In [ ]:
# 7. 이미지 분석에 사용할 데이터 셋이 최종 df입니다.

df_final = df[['category_id', 'thumbnail_link']].sort_values('category_id').reset_index(drop=True)

In [ ]:
df_final = df_final.loc[1974:].reset_index(drop=True)

In [ ]:
df_final_copy = df_final.copy()
# df_final = df_final_copy.copy()

In [ ]:
# category_id 별로 몇 개만 추출하여 학습할 것입니다.

df_final.category_id.value_counts()

Entertainment            52929
People & Blogs           33270
News & Politics          22182
Sports                   13543
Comedy                   10736
Music                    10515
Howto & Style             9743
Film & Animation          9226
Gaming                    5591
Education                 4083
Science & Technology      3472
Autos & Vehicles          3410
Pets & Animals            2326
Travel & Events            849
Shows                      409
Nonprofits & Activism       14
Movies                      12
Trailers                     3
Name: category_id, dtype: int64

In [ ]:
# 최종 데이터 셋 저장

df_final.to_csv('/content/drive/MyDrive/project/df_final.csv')
df_Analysis.to_csv('/content/drive/MyDrive/project/df_Analysis.csv')